# --- Day 23: Amphipod ---

In [1]:
from Quiz import *
from collections import Counter
from copy import deepcopy

### --- Part One ---

A group of amphipods https://en.wikipedia.org/wiki/Amphipoda notice your fancy submarine and flag you down. "With such an impressive shell," one amphipod <ins>says</ins>, "surely you can help us with a question that has stumped our best scientists."

They go on to explain that a group of timid, stubborn amphipods live in a nearby burrow. Four types of amphipods live there: __Amber__ (`A`), __Bronze__ (`B`), __Copper__ (`C`), and __Desert__ (`D`). They live in a burrow that consists of a __hallway__ and four __side rooms__. The side rooms are initially full of amphipods, and the hallway is initially empty.

They give you a __diagram of the situation__ (your puzzle input), including locations of each amphipod (`A`, `B`, `C`, or `D`, each of which is occupying an otherwise open space), walls (`#`), and open space (`.`).

For example:

`#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########`
  
The amphipods would like a method to organize every amphipod into side rooms so that each side room contains one type of amphipod and the types are sorted `A`-`D` going left to right, like this:

`#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########`
  
Amphipods can move up, down, left, or right so long as they are moving into an unoccupied open space. Each type of amphipod requires a different amount of __energy__ to move one step: Amber amphipods require `1` energy per step, Bronze amphipods require `10` energy, Copper amphipods require `100`, and Desert ones require `1000`. The amphipods would like you to find a way to organize the amphipods that requires the __least total energy__.

However, because they are timid and stubborn, the amphipods have some extra rules:

- Amphipods will never __stop on the space immediately outside any room__. They can move into that space so long as they immediately continue moving. (Specifically, this refers to the four open spaces in the hallway that are directly above an amphipod starting position.)
- Amphipods will never __move from the hallway into a room__ unless that room is their destination room __and__ that room contains no amphipods which do not also have that room as their own destination. If an amphipod's starting room is not its destination room, it can stay in that room until it leaves the room. (For example, an Amber amphipod will not move from the hallway into the right three rooms, and will only move into the leftmost room if that room is empty or if it only contains other Amber amphipods.)
- Once an amphipod stops moving in the hallway, __it will stay in that spot until it can move into a room__. (That is, once any amphipod starts moving, any other amphipods currently in the hallway are locked in place and will not move again until they can move fully into a room.)

In the above example, the amphipods can be organized using a minimum of __`12521`__ energy. One way to do this is shown below.

Starting configuration:

`#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########`
  
One Bronze amphipod moves into the hallway, taking 4 steps and using `40` energy:

`#############
#...B.......#
###B#C#.#D###
  #A#D#C#A#
  #########`
  
The only Copper amphipod not in its side room moves there, taking 4 steps and using 400 energy:

`#############
#...B.......#
###B#.#C#D###
  #A#D#C#A#
  #########`
  
A Desert amphipod moves out of the way, taking 3 steps and using 3000 energy, and then the Bronze amphipod takes its place, taking 3 steps and using `30` energy:

`#############
#.....D.....#
###B#.#C#D###
  #A#B#C#A#
  #########`
  
The leftmost Bronze amphipod moves to its room using `40` energy:

`#############
#.....D.....#
###.#B#C#D###
  #A#B#C#A#
  #########`
  
Both amphipods in the rightmost room move into the hallway, using `2003` energy in total:

`#############
#.....D.D.A.#
###.#B#C#.###
  #A#B#C#.#
  #########`
  
Both Desert amphipods move into the rightmost room using `7000` energy:

`#############
#.........A.#
###.#B#C#D###
  #A#B#C#D#
  #########`
  
Finally, the last Amber amphipod moves into its room, using `8` energy:

`#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########`
  
__What is the least energy required to organize the amphipods?__

In [2]:
COST = {'A':1 , 'B':10 , 'C':100 , 'D':1000}

In [3]:
#returns tuple of lists with occupants
def get_houses_1(dia):
    
    house_1 = []
    house_2 = []
    house_3 = []
    house_4 = []
    
    room_1 = dia[2].split("#")
    
    house_1.append(room_1[3])
    house_2.append(room_1[4])
    house_3.append(room_1[5])
    house_4.append(room_1[6])

    room_2 = dia[3].split("#")
    
    house_1.append(room_2[1])
    house_2.append(room_2[2])
    house_3.append(room_2[3])
    house_4.append(room_2[4])
        
    return (house_1 , house_2 , house_3 , house_4)

In [4]:
#returns true if all ocupants are in the right place
def done(state):

    houses = state[0]

    for k , v in houses.items():

        for ocup in v:

            if (ocup != k):

                return False

    return True

In [5]:
#returns true if k can move from col
def can_move_from(k , col):

    for ocup in col:

        if ((ocup != k) and (ocup != 'E')):

            return True

    return False

In [6]:
#returns true if k can move to col
def can_move_to(k , col):

    for ocup in col:

        if ((ocup != k) and (ocup != 'E')):

            return False

    return True

In [7]:
#returns the index of the corridor
def corridor_index(corridor):

    return {'A':2 , 'B':4 , 'C':6 , 'D':8}[corridor]

In [8]:
#returns the index of the first occupied house
def houses_index(col):
    
    i = 0

    for c in col:
        
        if (c != 'E'):

            return i
        
        i = i + 1

    return None

In [9]:
#returns the index of the last free destination
def destination_index(col):

    i = len(col)
    
    for c in reversed(col):

        i = i - 1
        
        if (c == 'E'):

            return i

    return None

In [10]:
#returns true if a is between the corridor and a house
def between(a , house , corridor):

    houses_corridor = (corridor_index(house) < a < corridor) 
    corridor_houses = (corridor < a < corridor_index(house))

    return ((houses_corridor) or (corridor_houses))

In [11]:
#returns true if there is a clear path between a and b
def clear_path(house , house_index , corridor):

    for i in range(len(corridor)):

        if (between(i , house , house_index) and (corridor[i] != 'E')):

            return False

    return True

In [12]:
#calculates the solution and returns the cost
def calculate_path(state):

    houses = state[0]
    corridor = state[1]
    
    key = (tuple((k , tuple(v)) for k , v in houses.items()) , tuple(corridor))

    if done(state):

        return 0

    if (key in DP):

        return DP[key]

    i = -1
    
    for c in corridor:
        
        i = i + 1

        if ((c in houses) and (can_move_to(c , houses[c]))):

            if (clear_path(c , i , corridor)):

                di = destination_index(houses[c])

                assert (di is not None)

                dist = di + 1 + abs(corridor_index(c) - i)
                cost = COST[c] * dist
                new_corridor = list(corridor)
                new_corridor[i] = 'E'
                corridor[i] = 'E'
                new_houses = deepcopy(houses)
                new_houses[c][di] = c

                return cost + calculate_path((new_houses , new_corridor))

    ans = 10000000

    for k , col in houses.items():

        if (not can_move_from(k , col)):

            continue

        ki = houses_index(col)

        if (ki is None):

            continue

        c = col[ki]

        for to_ in range(len(corridor)):

            if (to_ in [2, 4, 6, 8]):

                continue

            if (corridor[to_] != 'E'):

                continue

            if (clear_path(k , to_ , corridor)):

                dist = ki + 1 + abs(to_ - corridor_index(k))
                new_corridor = list(corridor)

                assert (new_corridor[to_] == 'E')

                new_corridor[to_] = c
                new_houses = deepcopy(houses)

                assert (new_houses[k][ki] == c)

                new_houses[k][ki] = 'E'
                ans = min(ans , COST[c] * dist + calculate_path((new_houses , new_corridor)))

    DP[key] = ans
    return ans

#### Test

In [13]:
houses = get_houses_1(test_diagram_1)

house_A = houses[0]
house_B = houses[1]
house_C = houses[2]
house_D = houses[3]

corridor = ['E'] * 11
start = ({'A':house_A , 'B':house_B , 'C':house_C , 'D':house_D} , corridor)
DP = {}

print("Minimum energy required:" , calculate_path(start))

Minimum energy required: 12521


#### Answer

In [14]:
houses = get_houses_1(diagram_1)

house_A = houses[0]
house_B = houses[1]
house_C = houses[2]
house_D = houses[3]

corridor = ['E'] * 11
start = ({'A':house_A , 'B':house_B , 'C':house_C , 'D':house_D} , corridor)
DP = {}

print("Minimum energy required:" , calculate_path(start))

Minimum energy required: 14510


-------------------------------------

### --- Part Two ---

As you prepare to give the amphipods your solution, you notice that the diagram they handed you was actually folded up. As you unfold it, you discover an extra part of the diagram.

Between the first and second lines of text that contain amphipod starting positions, insert the following lines:

`#D#C#B#A#
#D#B#A#C#`
  
So, the above example now becomes:

`#############
#...........#
###B#C#B#D###` <br>
  __` #D#C#B#A#`__ <br>
  __` #D#B#A#C#`__ <br>
  ` #A#D#C#A#
  #########`
  
The amphipods still want to be organized into rooms similar to before:

`#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########`
  
In this updated example, the least energy required to organize these amphipods is __`44169`__:

`#############
#...........#
###B#C#B#D###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########`

`#############
#..........D#
###B#C#B#.###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########`

`#############
#A.........D#
###B#C#B#.###
  #D#C#B#.#
  #D#B#A#C#
  #A#D#C#A#
  #########`

`#############
#A........BD#
###B#C#.#.###
  #D#C#B#.#
  #D#B#A#C#
  #A#D#C#A#
  #########`

`#############
#A......B.BD#
###B#C#.#.###
  #D#C#.#.#
  #D#B#A#C#
  #A#D#C#A#
  #########`

`#############
#AA.....B.BD#
###B#C#.#.###
  #D#C#.#.#
  #D#B#.#C#
  #A#D#C#A#
  #########`

`#############
#AA.....B.BD#
###B#.#.#.###
  #D#C#.#.#
  #D#B#C#C#
  #A#D#C#A#
  #########`

`#############
#AA.....B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#B#C#C#
  #A#D#C#A#
  #########`

`#############
#AA...B.B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#D#C#A#
  #########`

`#############
#AA.D.B.B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#.#C#A#
  #########`

`#############
#AA.D...B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#B#C#A#
  #########`

`#############
#AA.D.....BD#
###B#.#.#.###
  #D#.#C#.#
  #D#B#C#C#
  #A#B#C#A#
  #########`

`#############
#AA.D......D#
###B#.#.#.###
  #D#B#C#.#
  #D#B#C#C#
  #A#B#C#A#
  #########`

`#############
#AA.D......D#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#A#
  #########`

`#############
#AA.D.....AD#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#.#
  #########`

`#############
#AA.......AD#
###B#.#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#D#
  #########`

`#############
#AA.......AD#
###.#B#C#.###
  #D#B#C#.#
  #D#B#C#.#
  #A#B#C#D#
  #########`

`#############
#AA.......AD#
###.#B#C#.###
  #.#B#C#.#
  #D#B#C#D#
  #A#B#C#D#
  #########`

`#############
#AA.D.....AD#
###.#B#C#.###
  #.#B#C#.#
  #.#B#C#D#
  #A#B#C#D#
  #########`

`#############
#A..D.....AD#
###.#B#C#.###
  #.#B#C#.#
  #A#B#C#D#
  #A#B#C#D#
  #########`

`#############
#...D.....AD#
###.#B#C#.###
  #A#B#C#.#
  #A#B#C#D#
  #A#B#C#D#
  #########`

`#############
#.........AD#
###.#B#C#.###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########`

`#############
#..........D#
###A#B#C#.###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########`

`#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #A#B#C#D#
  #A#B#C#D#
  #########`
  
Using the initial configuration from the full diagram, __what is the least energy required to organize the amphipods?__

In [15]:
def get_houses_2(dia):
    
    house_1 = []
    house_2 = []
    house_3 = []
    house_4 = []
    
    room_1 = dia[2].split("#")
    
    house_1.append(room_1[3])
    house_2.append(room_1[4])
    house_3.append(room_1[5])
    house_4.append(room_1[6])

    room_2 = dia[3].split("#")
    
    house_1.append(room_2[1])
    house_2.append(room_2[2])
    house_3.append(room_2[3])
    house_4.append(room_2[4])
    
    room_3 = dia[4].split("#")
    
    house_1.append(room_3[1])
    house_2.append(room_3[2])
    house_3.append(room_3[3])
    house_4.append(room_3[4])
    
    room_4 = dia[5].split("#")
    
    house_1.append(room_4[1])
    house_2.append(room_4[2])
    house_3.append(room_4[3])
    house_4.append(room_4[4])
        
    return (house_1 , house_2 , house_3 , house_4)

#### Test

In [16]:
houses = get_houses_2(test_diagram_2)

house_A = houses[0]
house_B = houses[1]
house_C = houses[2]
house_D = houses[3]

corridor = ['E'] * 11
start = ({'A':house_A , 'B':house_B , 'C':house_C , 'D':house_D} , corridor)
DP = {}

print("Minimum energy required:" , calculate_path(start))

Minimum energy required: 44169


#### Answer

In [17]:
houses = get_houses_2(diagram_2)

house_A = houses[0]
house_B = houses[1]
house_C = houses[2]
house_D = houses[3]

corridor = ['E'] * 11
start = ({'A':house_A , 'B':house_B , 'C':house_C , 'D':house_D} , corridor)
DP = {}

print("Minimum energy required:" , calculate_path(start))

Minimum energy required: 49180
